In [2]:
import os

In [3]:
%pwd

'c:\\MBA\\Portfolio Projects\\Chicken-Disease-Detection\\research'

In [4]:
os.chdir("../") # changing to parent directory

In [5]:
%pwd

'c:\\MBA\\Portfolio Projects\\Chicken-Disease-Detection'

In [ ]:
# entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # key present in config.yaml file and creates the directory with artifacts name
        create_directories([self.config.artifacts_root]) 
 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            print("URL : self.config.source_URL : "+self.config.source_URL)
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    dt = data_ingestion.download_file()
    print("dt")
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-18 23:22:16,048: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-18 23:22:16,055: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-18 23:22:16,058: INFO: common: created directory at: artifacts]
[2025-09-18 23:22:16,059: INFO: common: created directory at: artifacts/data_ingestion]
URL : self.config.source_URL : https://github.com/mappy92/Data_Science_Projects_Data_Files/raw/main/Chicken-fecal-images.zip
[2025-09-18 23:22:17,177: INFO: 3026543807: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 64E0:CE913:471A70:5BFBE1: